In [10]:
import re

<link href="style.css" rel="stylesheet"></link>

<main>
<article class="day-desc"><h2>--- Day 10: Pipe Maze ---</h2><p>You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.</p>
<p>You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "<a href="https://en.wikipedia.org/wiki/Hot_spring" target="_blank">Hot Springs</a>" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.</p>
<p>The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.</p>
<p>Scanning the area, you discover that the entire field you're standing on is <span title="Manufactured by Hamilton and Hilbert Pipe Company">densely packed with pipes</span>; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).</p>
<p>The pipes are arranged in a two-dimensional grid of <em>tiles</em>:</p>
<ul>
<li><code>|</code> is a <em>vertical pipe</em> connecting north and south.</li>
<li><code>-</code> is a <em>horizontal pipe</em> connecting east and west.</li>
<li><code>L</code> is a <em>90-degree bend</em> connecting north and east.</li>
<li><code>J</code> is a <em>90-degree bend</em> connecting north and west.</li>
<li><code>7</code> is a <em>90-degree bend</em> connecting south and west.</li>
<li><code>F</code> is a <em>90-degree bend</em> connecting south and east.</li>
<li><code>.</code> is <em>ground</em>; there is no pipe in this tile.</li>
<li><code>S</code> is the <em>starting position</em> of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.</li>
</ul>
<p>Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is <em>one large, continuous loop</em>.</p>
<p>For example, here is a square loop of pipe:</p>
<pre><code>.....
.F-7.
.|.|.
.L-J.
.....
</code></pre>
<p>If the animal had entered this loop in the northwest corner, the sketch would instead look like this:</p>
<pre><code>.....
.<em>S</em>-7.
.|.|.
.L-J.
.....
</code></pre>
<p>In the above diagram, the <code>S</code> tile is still a 90-degree <code>F</code> bend: you can tell because of how the adjacent pipes connect to it.</p>
<p>Unfortunately, there are also many pipes that <em>aren't connected to the loop</em>! This sketch shows the same loop as above:</p>
<pre><code>-L|F7
7S-7|
L|7||
-L-J|
L|-JF
</code></pre>
<p>In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to <code>S</code>, pipes those pipes connect to, pipes <em>those</em> pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including <code>S</code>, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).</p>
<p>Here is a sketch that contains a slightly more complex main loop:</p>
<pre><code>..F7.
.FJ|.
SJ.L7
|F--J
LJ...
</code></pre>
<p>Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:</p>
<pre><code>7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
</code></pre>
<p>If you want to <em>get out ahead of the animal</em>, you should find the tile in the loop that is <em>farthest</em> from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps <em>along the loop</em> to reach from the starting point - regardless of which way around the loop the animal went.</p>
<p>In the first example with the square loop:</p>
<pre><code>.....
.S-7.
.|.|.
.L-J.
.....
</code></pre>
<p>You can count the distance each tile in the loop is from the starting point like this:</p>
<pre><code>.....
.012.
.1.3.
.23<em>4</em>.
.....
</code></pre>
<p>In this example, the farthest point from the start is <code><em>4</em></code> steps away.</p>
<p>Here's the more complex loop again:</p>
<pre><code>..F7.
.FJ|.
SJ.L7
|F--J
LJ...
</code></pre>
<p>Here are the distances for each tile on that loop:</p>
<pre><code>..45.
.236.
01.7<em>8</em>
14567
23...
</code></pre>
<p>Find the single giant loop starting at <code>S</code>. <em>How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?</em></p>
</article>

</main>


In [11]:
tests = [
    {
        "sketch": """
.....
.S-7.
.|.|.
.L-J.
.....
""",
        "expected": 4,
    },
    {
        "sketch": """
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
""",
        "expected": 8,
    },
    {
        "sketch": """
-L|F7
7S-7|
L|7||
-L-J|
L|-JF
""",
        "expected": 4,
    },
]

In [12]:
from cgi import test
from collections import deque
from itertools import product
from multiprocessing import process


class PipeMaze:
    translate = {"F": "┌", "7": "┐", "J": "┙", "L": "└", "|": "│", "-": "─"}

    def __init__(self, sketch: str) -> None:
        self.m = self.create_maze(sketch)
        self.start = self.get_start()
        self._replace_S()
        self.inside = set()
        self.clean_maze()

    def _replace_S(self) -> None:
        m = self.m
        r_s, c_s = self.start
        (r_fr, c_fr), (r_to, c_to) = sorted(
            self._neighbors(r_s, c_s), key=lambda p: (p[1], -p[0])
        )
        diff = r_to - r_fr, c_to - c_fr
        # . . . . . . .
        # . . 0 1 2 . .
        # . . ┌ ─ ┐ 0 .
        # . . │ . │ 1 .
        # . . └ ─ ┙ 2 .
        # . . . . . . .
        # . . . . . . .
        # "|"
        if diff == (-2, 0):  # (0, 0) - (2, 0) = (-2, 0)
            m[r_s][c_s] = "│"
        # "─"
        elif diff == (0, 2):  # (0, 2) - (0, 0) - (0, 0)
            m[r_s][c_s] = "─"
        # "┌"
        elif diff == (-1, 1):  # (0, 1) - (1, 0) = (-1, 1)
            m[r_s][c_s] = "┌"
        # "┐",
        elif diff == (1, 1):  # (1, 2) - (0, 1) = (1, 1)
            m[r_s][c_s] = "┐"
        # "┙"
        elif diff == (1, -1):  # (2, 1) - (1, 2) = (1, -1)
            m[r_s][c_s] = "┙"
        # "└"
        else:  # (1, 0) - (2, 1) = (-1, -11)
            m[r_s][c_s] = "└ "

    def get_loop(self) -> list[tuple[int, int]]:
        m = self.m
        queue = deque([(*self.start, 0)])
        loop = {}

        while queue:
            r, c, steps = queue.pop()
            if (r, c) in loop:
                continue
            loop[(r, c)] = 0
            queue.extend((rr, cc, steps + 1) for (rr, cc) in self._neighbors(r, c))

        return list(loop.keys())

    def maximum_steps(self) -> int:
        m = self.m
        queue = deque([(*self.start, 0)])
        max_steps = 0

        while queue:
            r, c, steps = queue.popleft()
            if m[r][c] not in "│└─┙┌┐":
                continue
            queue.extend((rr, cc, steps + 1) for (rr, cc) in self._neighbors(r, c))
            m[r][c] = str(steps)
            max_steps = max(max_steps, steps)

        return max_steps

    def get_start(self) -> tuple[int, int]:
        for r, c in product(range(len(self.m)), range(len(self.m[0]))):
            if self.m[r][c] == "S":
                return r, c

    def count_internal_tiles(self):
        loop = self.get_loop()
        loop.sort(key=lambda p: (p[1], p[0]))
        n = len(loop)

        prev_r, prev_c = loop[0]
        neigbors = self._neighbors(prev_r, prev_c)
        r, c = max(neigbors, key=lambda p: (p[1], -p[0]))
        self.left_right((prev_r, prev_c), (r, c))

        for i in range(n - 1):
            neigbors = self._neighbors(r, c)
            r_next, c_next = next(
                (r, c) for r, c in neigbors if (r, c) != (prev_r, prev_c)
            )
            prev_r, prev_c = r, c
            r, c = r_next, c_next
            self.left_right((prev_r, prev_c), (r, c))

        m = self.m
        rows, cols = len(m), len(m[0])
        queue = deque(list(self.inside))
        processed = set()
        while queue:
            r, c = queue.popleft()
            if m[r][c] == "." and not (r, c) in processed:
                processed.add((r, c))
                queue.extend(
                    (r + dr, c + dc)
                    for dr, dc in ((-1, 0), (0, 1), (1, 0), (0, -1))
                    if 0 <= r + dr < rows and 0 <= c + dc < cols
                )

        return len(processed)

    def left_right(self, p1: tuple[int, int], p2: tuple[int:int]) -> None:
        m = self.m
        diff = tuple(v2 - v1 for v1, v2 in zip(p1, p2))
        r, c = p1
        below, above = r + 1, r - 1
        left, right = c - 1, c + 1

        if diff == (-1, 0):  # UP
            if m[r][c] == "│":
                if m[r][right] == ".":
                    self.inside.add((r, right))
            elif m[r][c] == "┙":
                if m[below][c] == ".":
                    self.inside.add((below, c))
                if m[below][right] == ".":
                    self.inside.add((below, right))
                if m[r][right] == ".":
                    self.inside.add(((r, right)))
        elif diff == (0, 1):  # RIGHT
            if m[r][c] == "─":
                if m[below][c] == ".":
                    self.inside.add((below, c))
        elif diff == (1, 0):  # DOWN
            if m[r][c] == "│":
                if m[r][left] == ".":
                    self.inside.add((r, c - 1))
            elif m[r][c] == "┙":
                if m[r][right] == ".":
                    self.inside.add(((r, right)))
                if m[below][right] == ".":
                    self.inside.add((below, right))
                if m[below][c] == ".":
                    self.inside.add((below, c))
        else:  # LEFT
            # (1, 0), (-1, 0)
            if m[r][c] == "─":
                if m[above][c] == ".":
                    self.inside.add((above, c))
            elif m[r][c] == "┐":
                if m[r][right] == ".":
                    self.inside.add((r, right))
                if m[above][right] == ".":
                    self.inside.add((above, right))
                if m[above][c] == ".":
                    self.inside.add((above, c))

    def _neighbors(self, row: int, col: int) -> list[tuple[int, int]]:
        m = self.m
        rows, cols = len(m), len(m[0])
        c = m[row][col]

        neighbors = []

        if row + 1 < rows and c in "S┌┐│" and m[row + 1][col] in "│└┙S":
            neighbors.append((row + 1, col))
        if col + 1 < cols and c in "S┌└─" and m[row][col + 1] in "─┙┐S":
            neighbors.append((row, col + 1))
        if col - 1 > -1 and c in "S┐┙─" and m[row][col - 1] in "─└┌S":
            neighbors.append((row, col - 1))
        if row - 1 > -1 and c in "S┙└│" and m[row - 1][col] in "│┐┌S":
            neighbors.append((row - 1, col))

        return neighbors

    def create_maze(self, sketch: str):
        m = [["."] + list(l) + ["."] for l in sketch.strip().splitlines()]
        m.insert(0, ["n"] * len(m[0]))
        m.append(["n"] * len(m[0]))
        rows, cols = len(m), len(m[0])
        for r, c in product(range(rows), range(cols)):
            m[r][c] = self.translate.get(m[r][c], str(m[r][c]))
        return m

    def clean_maze(self):
        m = self.m
        rows, cols = len(m), len(m[0])
        loop = set(self.get_loop())
        for r, c in product(range(rows), range(cols)):
            if m[r][c] not in ".":
                if (r, c) not in loop:
                    m[r][c] = "."

    def __str__(self) -> str:
        return "\n".join(" ".join(str(l) for l in row) for row in self.m)


assert all(
    PipeMaze(test["sketch"]).maximum_steps() == test["expected"] for test in tests
)

In [13]:
sketch_1 = """
.....
.S-7.
.|.|.
.L-J.
.....
"""
assert PipeMaze(sketch_1).maximum_steps() == 4

sketch_2 = """
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
"""
assert PipeMaze(sketch_2).maximum_steps() == 8

In [14]:
with open("./input/day10.txt") as f:
    print(PipeMaze(f.read()).maximum_steps())

7012


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>7012</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is <em>within the area enclosed by the loop</em>?</p>
<p>To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:</p>
<pre><code>...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
</code></pre>
<p>The above loop encloses merely <em>four tiles</em> - the two pairs of <code>.</code> in the southwest and southeast (marked <code>I</code> below). The middle <code>.</code> tiles (marked <code>O</code> below) are <em>not</em> in the loop. Here is the same loop again with those regions marked:</p>
<pre><code>...........
.S-------7.
.|F-----7|.
.||<em>OOOOO</em>||.
.||<em>OOOOO</em>||.
.|L-7<em>O</em>F-J|.
.|<em>II</em>|<em>O</em>|<em>II</em>|.
.L--J<em>O</em>L--J.
.....<em>O</em>.....
</code></pre>
<p>In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, <code>I</code> is still within the loop and <code>O</code> is still outside the loop:</p>
<pre><code>..........
.S------7.
.|F----7|.
.||<em>OOOO</em>||.
.||<em>OOOO</em>||.
.|L-7F-J|.
.|<em>II</em>||<em>II</em>|.
.L--JL--J.
..........
</code></pre>
<p>In both of the above examples, <code><em>4</em></code> tiles are enclosed by the loop.</p>
<p>Here's a larger example:</p>
<pre><code>.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
</code></pre>
<p>The above sketch has many random bits of ground, some of which are in the loop (<code>I</code>) and some of which are outside it (<code>O</code>):</p>
<pre><code><em>O</em>F----7F7F7F7F-7<em>OOOO</em>
<em>O</em>|F--7||||||||FJ<em>OOOO</em>
<em>O</em>||<em>O</em>FJ||||||||L7<em>OOOO</em>
FJL7L7LJLJ||LJ<em>I</em>L-7<em>OO</em>
L--J<em>O</em>L7<em>III</em>LJS7F-7L7<em>O</em>
<em>OOOO</em>F-J<em>II</em>F7FJ|L7L7L7
<em>OOOO</em>L7<em>I</em>F7||L7|<em>I</em>L7L7|
<em>OOOOO</em>|FJLJ|FJ|F7|<em>O</em>LJ
<em>OOOO</em>FJL-7<em>O</em>||<em>O</em>||||<em>OOO</em>
<em>OOOO</em>L---J<em>O</em>LJ<em>O</em>LJLJ<em>OOO</em>
</code></pre>
<p>In this larger example, <code><em>8</em></code> tiles are enclosed by the loop.</p>
<p>Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:</p>
<pre><code>FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
</code></pre>
<p>Here are just the tiles that are <em>enclosed by the loop</em> marked with <code>I</code>:</p>
<pre><code>FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ<em>I</em>F7FJ-
L---JF-JLJ<em>IIII</em>FJLJJ7
|F|F-JF---7<em>III</em>L7L|7|
|FFJF7L7F-JF7<em>II</em>L---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
</code></pre>
<p>In this last example, <code><em>10</em></code> tiles are enclosed by the loop.</p>
<p>Figure out whether you have time to search for the nest by calculating the area within the loop. <em>How many tiles are enclosed by the loop?</em></p>
</article>
</main>


In [15]:
tests_II = [
    {
        "sketch": """
...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
""",
        "expected": 4,
    },
    {
        "sketch": """
..........
.S------7.
.|F----7|.
.||....||.
.||....||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
..........
""",
        "expected": 4,
    },
    {
        "sketch": """
..........
.S------7.
.|F----7|.
.||....||.
.||....||.
.||--F-J|.
.|L-7L--J.
.L--J.....
..........
""",
        "expected": 0,
    },
    {
        "sketch": """
..........
.S--7F--7.
.|..||..|.
.|F-JL-7|.
.||....||.
.|L----J|.
.|......|.
.L------J.
..........
""",
        "expected": 10,
    },
    {
        "sketch": """
..........
.S------7.
.|.F--7.|.
.|.|..L-J.
.|.|..F-7.
.|.L--J.|.
.|......|.
.L------J.
..........
""",
        "expected": 12,
    },
    {
        "sketch": """
..........
.S------7.
.|F---7.|.
.LJ...|.|.
.F7...|.|.
.|L---J.|.
.|......|.
.L------J.
..........
""",
        "expected": 10,
    },
    {
        "sketch": """
..........
.S------7.
.|......|.
.|......|.
.|......|.
.|......|.
.|......|.
.L------J.
..........
""",
        "expected": 30,
    },
    {
        "sketch": """
.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
""",
        "expected": 8,
    },
    {
        "sketch": """
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
""",
        "expected": 10,
    },
    {
        "sketch": """
7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
""",
        "expected": 1,
    },
    {
        "sketch": """
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
""",
        "expected": 1,
    },
]

# 👻👻👻👻👻👻👻👻👻👻👻👻👻👻👻👻👻👻👻👻👻👻👻👻

In [16]:
for i, test in enumerate(tests_II):
    print(f"{i=}")
    sketch, expected = test.values()
    pm = PipeMaze(sketch)
    print(pm)
    print()

assert all(
    PipeMaze(test["sketch"]).count_internal_tiles() == test["expected"]
    for test in tests_II
)

i=0
. . . . . . . . . . . . .
. . . . . . . . . . . . .
. . ┌ ─ ─ ─ ─ ─ ─ ─ ┐ . .
. . │ ┌ ─ ─ ─ ─ ─ ┐ │ . .
. . │ │ . . . . . │ │ . .
. . │ │ . . . . . │ │ . .
. . │ └ ─ ┐ . ┌ ─ ┙ │ . .
. . │ . . │ . │ . . │ . .
. . └ ─ ─ ┙ . └ ─ ─ ┙ . .
. . . . . . . . . . . . .
. . . . . . . . . . . . .

i=1
. . . . . . . . . . . .
. . . . . . . . . . . .
. . ┌ ─ ─ ─ ─ ─ ─ ┐ . .
. . │ ┌ ─ ─ ─ ─ ┐ │ . .
. . │ │ . . . . │ │ . .
. . │ │ . . . . │ │ . .
. . │ └ ─ ┐ ┌ ─ ┙ │ . .
. . │ . . │ │ . . │ . .
. . └ ─ ─ ┙ └ ─ ─ ┙ . .
. . . . . . . . . . . .
. . . . . . . . . . . .

i=2
. . . . . . . . . . . .
. . . . . . . . . . . .
. . ┌ ─ ─ ─ ─ ─ ─ ┐ . .
. . │ ┌ ─ ─ ─ ─ ┐ │ . .
. . │ │ . . . . │ │ . .
. . │ │ . . . . │ │ . .
. . │ │ . . ┌ ─ ┙ │ . .
. . │ └ ─ ┐ └ ─ ─ ┙ . .
. . └ ─ ─ ┙ . . . . . .
. . . . . . . . . . . .
. . . . . . . . . . . .

i=3
. . . . . . . . . . . .
. . . . . . . . . . . .
. . ┌ ─ ─ ┐ ┌ ─ ─ ┐ . .
. . │ . . │ │ . . │ . .
. . │ ┌ ─ ┙ └ ─ ┐ │ . .
. . │ │ . . . . │ │ . .
. . │ └ ─ ─ ─ ─ ┙ │ . .

In [17]:
# with open("./input/day10.txt") as f:
#     print(determine_enclosed_tiles(f.read()))

# #  613 to high

In [18]:
with open("./input/day10.txt") as f:
    pm = PipeMaze(f.read())
    # lp = pm.get_loop()
    # for i, (r, c) in enumerate(lp):
    #     pm.m[r][c] = str(i)
    pm.count_internal_tiles()
    print(pm.count_internal_tiles())

395


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>395</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2023">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="10/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Pipe+Maze%22+%2D+Day+10+%2D+Advent+of+Code+2023&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F10&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Pipe+Maze%22+%2D+Day+10+%2D+Advent+of+Code+2023+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F10';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
